In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization

# Function to load images and labels from the dataset folders
def load_dataset(dataset_path):
    authentic_path = os.path.join(dataset_path, 'authentic')
    tampered_path = os.path.join(dataset_path, 'forged')

    authentic_images = []
    tampered_images = []
    labels = []

    # Load authentic images
    # authentic_count = 0
    for filename in os.listdir(authentic_path):
        # if authentic_count >= 5000:
        #     break
        img = cv2.imread(os.path.join(authentic_path, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            authentic_images.append(img)
            labels.append(0)  # 0 for authentic
            # authentic_count += 1

    # Load tampered images
    # tampered_count = 0
    for filename in os.listdir(tampered_path):
        # if tampered_count >= 5000:
        #     break
        img = cv2.imread(os.path.join(tampered_path, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            tampered_images.append(img)
            labels.append(1)  # 1 for tampered
            # tampered_count += 1

    return authentic_images, tampered_images, labels
# Load dataset
authentic_images, tampered_images, labels = load_dataset('/kaggle/input/all-forged-authentic')

# Combine authentic and tampered images
X = np.array(authentic_images + tampered_images)
y = np.array(labels)



In [ ]:
# Split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("Authentic: ",len(authentic_images))
print("Forged: ",len(tampered_images))

**Below is the resnet code**

In [ ]:
# Load pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

# for layer in base_model.layers[:-40]:
#     layer.trainable = False

# Create a new model on top of the ResNet50 base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)), 
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  
    Dense(1, activation='sigmoid')  # Binary classification: forged or authentic
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with learning rate scheduler
model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Make predictions on validation data
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# Classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

In [ ]:
# Calculate AUC
auc_score = roc_auc_score(y_val, y_pred_proba)
print("AUC Score:", auc_score)

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
image_path = '/kaggle/input/testing/DSCN45tamp237.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))  # Resize to match the input shape expected by ResNet50
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (since ResNet50 expects RGB)
image = np.expand_dims(image, axis=0)
    
prediction = model.predict(image)
print(prediction[0][0])

if prediction[0][0] > 0.5:
    print("Forged Image")
else:
    print("Authentic Image")

In [ ]:
model.summary()

**Below is the DenseNet Code**

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers
import tensorflow as tf


# Load pre-trained DenseNet121 model without the top classification layer
base_denmodel = DenseNet121(weights='imagenet', include_top=False)

# for layer in base_model.layers[:-60]:
#     layer.trainable = False

# Create a new model on top of the DenseNet121 base model
den_model = Sequential([
    base_denmodel,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)), 
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  
    Dense(1, activation='sigmoid')  # Binary classification: forged or authentic
])

# Compile the model
den_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


den_model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = den_model.evaluate(X_val, y_val)
print("Test Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Make predictions on validation data
y_pred_proba = den_model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# Classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

In [ ]:
# Calculate AUC
auc_score = roc_auc_score(y_val, y_pred_proba)
print("AUC Score:", auc_score)

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc_score))
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
den_model.summary()